# Fine tuning on pre-trained keras model InceptionV3

In [82]:
import os 

import numpy as np
from matplotlib import pyplot
import tensorflow as tf
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.optimizers import SGD
from keras import callbacks
from sklearn.metrics import confusion_matrix, accuracy_score
from matplotlib import pyplot
import seaborn as sns


In [27]:
path = "/home/ubuntu/nbs/data/seedlings"
train_path = os.path.join(path, "train")
valid_path = os.path.join(path, "valid")
test_path = os.path.join(path, "test")
preview_path = os.path.join(path, "preview")
checkpoint_path = os.path.join(path, "best_weights.hdf5")

### Data generator with augmentation

In [20]:
# Get batches from the dirname directory, perform augmentation (rotation, shift, flip, zoom) 
# and return batch genarator object.
def get_batches(dirname, shuffle=True, batch_size=4, class_mode='categorical',
                target_size=(224,224)):
    gen = image.ImageDataGenerator(
                        rotation_range=180, 
                        width_shift_range=0.1, height_shift_range=0.1, 
                        horizontal_flip=True, vertical_flip=True,
                        zoom_range=0.2,
                        preprocessing_function=preprocess_input
                        )
    flow = gen.flow_from_directory(
                        dirname, 
                        shuffle=True,
                        batch_size=batch_size, 
                        class_mode='categorical',
                        target_size=target_size,
                        )

    return flow 

### Take a look at augmented images

In [34]:
#Take a look at augmented images
gen = image.ImageDataGenerator(
                        rotation_range=180, 
                        width_shift_range=0.1, height_shift_range=0.1, 
                        horizontal_flip=True, vertical_flip=True,
                        zoom_range=0.2,
                        preprocessing_function=preprocess_input
                        )

img = image.load_img(os.path.join(train_path, 'Charlock/0a7e1ca41.png'))
x = image.img_to_array(img)
x = x.reshape((1,) + x.shape)

# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `preview/` directory
for z in range(9):
    batch = next(gen.flow(x, save_to_dir=preview_path, save_prefix='Black_grass', save_format='jpeg'))
    
for i, pic in enumerate(os.listdir(preview_path)[:9]):
    img = image.load_img(os.path.join(preview_path, pic))
    pyplot.subplot(330 + 1 + i)
    pyplot.imshow(img)
pyplot.show()

### Updating InceptionV3 model

In [21]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer (12 classes)
predictions = Dense(12, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# train only the top layers
# freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

### Creating callbacks (early stopping and checkpoint)

In [22]:
# Early stop training if loss starts to increase duaring 2 epochs. Check point best models.
callbacks = [
    callbacks.EarlyStopping(monitor='val_loss', patience=2, verbose=0),
    callbacks.ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True, verbose=0),
]

### Train top level of model 

In [23]:
# get train and valid augmented data generators
batch_size = 64
train_batches = get_batches(train_path, batch_size=batch_size)
valid_batches = get_batches(valid_path, batch_size=batch_size)
# train model
model.fit_generator(train_batches,
                    epochs=30, 
                    validation_data=valid_batches, 
                    steps_per_epoch=10000/batch_size, 
                    validation_steps=2500/batch_size,
                    callbacks=callbacks,
                   )

Found 3070 images belonging to 12 classes.
Found 840 images belonging to 12 classes.
Epoch 1/30
156/156 [==============================] - 271s 2s/step - loss: 2.3260 - acc: 0.3964 - val_loss: 2.0876 - val_acc: 0.3951
Epoch 2/30
156/156 [==============================] - 228s 1s/step - loss: 1.2880 - acc: 0.5765 - val_loss: 1.3981 - val_acc: 0.5138
Epoch 3/30
156/156 [==============================] - 230s 1s/step - loss: 1.0664 - acc: 0.6468 - val_loss: 1.3249 - val_acc: 0.5612
Epoch 4/30
156/156 [==============================] - 232s 1s/step - loss: 0.9739 - acc: 0.6718 - val_loss: 1.2465 - val_acc: 0.5755
Epoch 5/30
156/156 [==============================] - 231s 1s/step - loss: 0.8955 - acc: 0.6955 - val_loss: 1.1040 - val_acc: 0.6321
Epoch 6/30
156/156 [==============================] - 228s 1s/step - loss: 0.8505 - acc: 0.7167 - val_loss: 0.9689 - val_acc: 0.6762
Epoch 7/30
156/156 [==============================] - 230s 1s/step - loss: 0.8073 - acc: 0.7260 - val_loss: 1.0085 - 

### Fine-tuning convolutional layers

In [44]:
# top layers are well trained, can start fine-tuning 
print("Number of InceptionV3 layers {}".format(len(base_model.layers)))
# freeze the first 249 layers and unfreeze the rest
for layer in model.layers[:249]:
    layer.trainable = False
for layer in model.layers[249:]:
    layer.trainable = True

#recompile the model with a low learning rate
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['acc'])


# Callbacks (early stop training if loss starts to increase duaring 2 epochs. Check point best models.)
callbacks = [
    callbacks.EarlyStopping(monitor='val_loss', patience=2, verbose=0),
    callbacks.ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True, verbose=0),
]
# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit_generator(train_batches, 
                    epochs=30, 
                    validation_data=valid_batches, 
                    callbacks=callbacks, 
                    steps_per_epoch=10000/batch_size, 
                    validation_steps=2500/batch_size)

Number of InceptionV3 layers 311
Epoch 1/30
156/156 [==============================] - 243s 2s/step - loss: 0.5965 - acc: 0.7989 - val_loss: 0.7755 - val_acc: 0.7404
Epoch 2/30
156/156 [==============================] - 231s 1s/step - loss: 0.5441 - acc: 0.8122 - val_loss: 0.7960 - val_acc: 0.7383
Epoch 3/30
156/156 [==============================] - 233s 1s/step - loss: 0.5235 - acc: 0.8228 - val_loss: 0.7334 - val_acc: 0.7479
Epoch 4/30
156/156 [==============================] - 236s 2s/step - loss: 0.4948 - acc: 0.8311 - val_loss: 0.7092 - val_acc: 0.7538
Epoch 5/30
156/156 [==============================] - 234s 2s/step - loss: 0.4656 - acc: 0.8381 - val_loss: 0.6738 - val_acc: 0.7605
Epoch 6/30
156/156 [==============================] - 231s 1s/step - loss: 0.4540 - acc: 0.8451 - val_loss: 0.6384 - val_acc: 0.7764
Epoch 7/30
156/156 [==============================] - 231s 1s/step - loss: 0.4353 - acc: 0.8503 - val_loss: 0.6533 - val_acc: 0.7664
Epoch 8/30
156/156 [================

In [46]:
scores = model.evaluate_generator(valid_batches, steps=2500/batch_size)
for i,s in enumerate(scores):
    print("{} {}".format(model.metrics_names[i], scores[i]))


loss 0.637873838912
acc 0.777265100671
